In [7]:
# Here we take care of paths.
# Make sure root project directory is named 'VESUVIUS_Challenge' for this to work

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in the root folder of the project
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge
Current path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge


In [8]:
import torch
import monai
from monai.visualize import matshow3d
#import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from typing import Tuple, List
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Vesuvius_Dataset import Vesuvius_Tile_Datamodule
from lit_models.Vesuvius_Lit_Model import Lit_Model
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.nn as nn
import torchvision
#from Models.pvtv2 import PyramidVisionTransformerV2
from timm.models.pvt_v2 import PyramidVisionTransformerV2


In [9]:
PATCH_SIZE = 256
Z_DIM = 16
COMPETITION_DATA_DIR_str =  "kaggle/input/vesuvius-challenge-ink-detection/"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")

# change to the line below if not using Apple's M1 or chips
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model = PyramidVisionTransformerV2()

In [ ]:
class Model(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        self.pre_model =monai.networks.nets.UNet(
                                        spatial_dims=3,
                                        in_channels=1,
                                        out_channels=1,
                                        channels=( 32, 64,128),
                                        strides=(2, 2,),
                                        num_res_units=2,
                                        dropout=0,
                                        norm='instance',
                                        bias =False,

                                    )
        
        self.model =monai.networks.nets.SwinUNETR(img_size = (256,256),
                                                      in_channels = 16 ,
                                                      out_channels = 1 ,
                                                      depths=(2, 2, 2, 2),
                                                      num_heads=(3, 6, 12, 24),
                                                      feature_size=24,
                                                      norm_name='instance',
                                                      drop_rate=0.0,
                                                      attn_drop_rate=0.0,
                                                      dropout_path_rate=0.0,
                                                      normalize=True,
                                                      use_checkpoint=False,
                                                      spatial_dims=2,
                                                      downsample='mergingv2')


        
    def forward(self, x):
        #x = self.pre_model(x)
        x = x.squeeze(1)
        out = self.model(x)
        return out
    
    
